In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.datasets import mnist
import numpy as np


In [3]:

(X_train, y_train), (X_valid, y_valid) = mnist.load_data()


X_train, X_valid = X_train / 255.0, X_valid / 255.0


In [4]:
model = Sequential([ 
    Input(shape=(28, 28)), 
    Flatten(),  
    Dense(128, activation='relu'),  
    Dense(64, activation='relu'), 
    Dense(10, activation='softmax')]) 

In [5]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=keras.optimizers.SGD(learning_rate=1e-3), 
              metrics=["accuracy"])

In [6]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.2933 - loss: 2.1511 - val_accuracy: 0.7470 - val_loss: 1.4169
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7726 - loss: 1.2045 - val_accuracy: 0.8283 - val_loss: 0.7421
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8295 - loss: 0.7043 - val_accuracy: 0.8616 - val_loss: 0.5455
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8610 - loss: 0.5409 - val_accuracy: 0.8788 - val_loss: 0.4573
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8773 - loss: 0.4604 - val_accuracy: 0.8880 - val_loss: 0.4062
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8874 - loss: 0.4174 - val_accuracy: 0.8960 - val_loss: 0.3731
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8943 - loss: 0.3833 - val_accuracy: 0.9018 - val_loss: 0.3501
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8990 - loss: 0.3609 - 

In [7]:
import keras_tuner as kt

In [22]:
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(28, 28)))
    model.add(Flatten())
    
    # Tune the number of layers
    for i in range(hp.Int('num_layers', 1, 5)):  # Increased from 1-3 to 1-5
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation='relu'))  # Increased max units from 256 to 512
    
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-1, sampling='LOG')),  # Adjusted max learning rate
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [23]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Increased from 10 to 20
    executions_per_trial=1,
    directory='my_dir',
    project_name='mnist_tuning_98'
)


In [24]:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))


Trial 20 Complete [00h 02m 21s]
val_accuracy: 0.9801999926567078

Best val_accuracy So Far: 0.9854999780654907
Total elapsed time: 00h 40m 04s


In [25]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [26]:
model = tuner.hypermodel.build(best_hps)

In [27]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8088 - loss: 0.5927 - val_accuracy: 0.9527 - val_loss: 0.1521
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9659 - loss: 0.1107 - val_accuracy: 0.9648 - val_loss: 0.1140
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9767 - loss: 0.0748 - val_accuracy: 0.9726 - val_loss: 0.0868
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9836 - loss: 0.0510 - val_accuracy: 0.9750 - val_loss: 0.0865
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9869 - loss: 0.0390 - val_accuracy: 0.9798 - val_loss: 0.0632
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9920 - loss: 0.0266 - val_accuracy: 0.9706 - val_loss: 0.0987
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9931 - loss: 0.0232 - val_accuracy: 0.9805 - val_loss: 0.0710
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9948 - loss: 0.0173 - 

In [28]:
val_loss, val_accuracy = model.evaluate(X_valid, y_valid) 
print(f"Validation loss: {val_loss}") 
print(f"Validation accuracy: {val_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9785 - loss: 0.1037
Validation loss: 0.08360625803470612
Validation accuracy: 0.9825999736785889


In [29]:
(X_test, y_test) = mnist.load_data()[1] 
X_test = X_test / 255.0

In [30]:
import time
start_time = time.time() 
test_loss, test_accuracy = model.evaluate(X_test, y_test) 
end_time = time.time() 
training_time = end_time - start_time

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9785 - loss: 0.1037


In [31]:
print(f"Test loss: {test_loss}") 
print(f"Test accuracy: {test_accuracy}") 
print(f"Training time: {training_time:.2f} seconds")

Test loss: 0.08360625803470612
Test accuracy: 0.9825999736785889
Training time: 0.80 seconds
